In [1]:
import os
import sys
import re
import pandas as pd
import commands
import ast
from StringIO import StringIO
import itertools
import pyspark.sql.functions
from pyspark.sql.functions import col
from datetime import datetime
from collections import Counter
import re
import numpy as np
#from pyspark.sql import SparkSession
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
if not os.path.isdir(spark_home):
    raise ValueError('SPARK_HOME environment variable is not a directory')
if not os.path.isdir(os.path.join(spark_home, 'python')):
    raise ValueError('SPARK_HOME directory does not contain python')
sys.path.insert(0, os.path.join(spark_home, 'python'))
pylib_list = (item for item in os.listdir(os.path.join(spark_home, 'python/lib/'))
              if re.match(r'py4j-\d+(\.\d+)+-src\.zip\Z', item)
              )
try:
    py4j_file = max(pylib_list)
    py4j = os.path.join(spark_home, os.path.join('python/lib', py4j_file))
except ValueError:
    raise ValueError(
        'Could not find py4j'
    )
sys.path.insert(0, py4j)

spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark" in  open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", " --master yarn-client \
                                         --executor-memory 4g --executor-cores 5 --driver-memory 16g"
                                        )
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

with open(os.path.join(spark_home, 'python/pyspark/shell.py')) as f:
    code = compile(f.read(), os.path.join(spark_home, 'python/pyspark/shell.py'), 'exec')
    exec(code)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.0.2.6.5.0-292
      /_/

Using Python version 2.7.15 (default, May  1 2018 23:32:55)
SparkSession available as 'spark'.


In [74]:
preprod=pd.read_csv("/s/latluri1/DES/DESPRE-Prod_processed.csv", header=None,delimiter="|").apply(lambda x: x.astype(str).str.upper())
DESDEV=pd.read_csv("/s/latluri1/DES/DESDEV_processed.csv", header=None,delimiter="|").apply(lambda x: x.astype(str).str.upper())
DESDSC_Jan25=pd.read_csv("/s/latluri1/DES/DESDSC_Jan25_processed.csv", header=None,delimiter="|").apply(lambda x: x.astype(str).str.upper())
DESQA=pd.read_csv("/s/latluri1/DES/DESQA_processed.csv", header=None,delimiter="|").apply(lambda x: x.astype(str).str.upper())
prod=pd.read_csv("/s/latluri1/DES/DES_prod_processed.csv", header=None,delimiter="|").apply(lambda x: x.astype(str).str.upper())


In [76]:
old_ddl=DESDSC_Jan25
new_ddl=DESDEV
new_ddl.name='DESDEV'
output=''
output=output+"Table\n"
# tables removed
for tab_col in set(list(new_ddl[0])).difference(set(list(old_ddl[0]))):
    output=output+ str(tab_col)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
b

,Table
0,GCCT_CASECLASSIFICATION_TEAM
1,GCCT_MARKETINGPROGRAM
2,MSDYN_QUESTION_ANSWER_N_N
3,GCCT_GCCT_SYMPTOMCLASSIFICATION_INCIDENT
4,GCCT_CASEDISPOSITIONTYPE_TEAM
5,GCCT_CASEDISPOSITIONTYPE_CASECLASSIFICATION
6,GCCT_TEAM_TEAM_CANASSIGNTOTEAM
7,GCCT_QAINCIDENT
8,GCCT_TEAM_SYSTEMUSER
9,CALENDAR


In [55]:
old_ddl=DESDSC_Jan25
new_ddl=DESDEV
new_ddl.name='DESDEV'

output=''
output=output+"Table\n"
# tables removed
for tab_col in set(list(new_ddl[0])).difference(set(list(old_ddl[0]))):
    output=output+ str(tab_col)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
writer = pd.ExcelWriter("Schemachanges"+new_ddl.name+".xlsx", engine = 'xlsxwriter')
b.to_excel(writer, sheet_name=new_ddl.name+"_Newfields", index=False)
del a
del b
del output

output=''
output=output+"Table\n"
# tables added
for tab_col in set(list(old_ddl[0])).difference(set(list(new_ddl[0]))):
    output=output+ str(tab_col)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
writer = pd.ExcelWriter("Schemachanges"+new_ddl.name+".xlsx", engine = 'xlsxwriter')
b.to_excel(writer, sheet_name=new_ddl.name+"_Newfields", index=False)
del a
del b
del output

output=''
output=output+"Table|Column|DType\n"
#New fields in common tables
for i in set(new_ddl[0]).difference(set(old_ddl[0])):
    tab_col=new_ddl[new_ddl[0] == i].values[0][0]
    dtype=new_ddl[new_ddl[0] == i].values[0][1]
    output=output+ str(tab_col).replace("-","|").strip().replace(" ","|")+"|"+str(dtype)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
writer = pd.ExcelWriter("Schemachanges"+new_ddl.name+".xlsx", engine = 'xlsxwriter')
b.to_excel(writer, sheet_name=new_ddl.name+"_Newfields", index=False)
del a
del b
del output
output=''
output=output+"Table|Column|DType\n"
#Fields  removed in common tables
for i in set(old_ddl[0]).difference(set(new_ddl[0])):
    tab_name=old_ddl[old_ddl[0] == i].values[0][0]
    dtype=old_ddl[old_ddl[0] == i].values[0][1]
    output=output+str(tab_name).replace("-","|").strip().replace(" ","|")+"|"+str(dtype)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
b.to_excel(writer, sheet_name=new_ddl.name+"_removedfields", index=False)
del a
del b
del output

output=''
output=output+"Table|Column|DType\n"
#Dtype changes
for i in set(old_ddl[0]).intersection(set(new_ddl[0])):
    #print i
    old=old_ddl[old_ddl[0] == i].values[0][1]
    new=new_ddl[new_ddl[0] == i].values[0][1]
    if old!=new:
        output=output+str(prod[prod[0] == i].values[0][0]).replace("-","|").strip().replace(" ","|")+"|"+old+"|"+new+"\n"


a=StringIO(output)
b=pd.read_csv(a, sep="|")
b.to_excel(writer, sheet_name=new_ddl.name+"_data_type_changes", index=False)
del a
del b
del output

In [56]:
new_ddl

,0,1
0,ACCOUNT-ACCOUNT_IDENTITY,INT
1,ACCOUNT-ACCOUNT_ROWVERSION,TIMESTAMP
2,ACCOUNT-ID,UNIQUEIDENTIFIER
3,ACCOUNT-SINKCREATEDON,DATETIME
4,ACCOUNT-SINKMODIFIEDON,DATETIME
5,ACCOUNT-STATECODE,INT
6,ACCOUNT-STATUSCODE,INT
7,ACCOUNT-CUSTOMERTYPECODE,INT
8,ACCOUNT-ADDRESS1_FREIGHTTERMSCODE,INT
9,ACCOUNT-ADDRESS1_ADDRESSTYPECODE,INT


In [23]:
old_ddl=DESDSC_Jan25
new_ddl=prod
output=''
output=output+"Table|Column|DType\n"
#Fields and tables removed
for i in set(old_ddl[0]).difference(set(new_ddl[0])):
    tab_name=old_ddl[old_ddl[0] == i].values[0][0]
    dtype=old_ddl[old_ddl[0] == i].values[0][1]
    print str(tab_name).replace("-","|").strip().replace(" ","|")+"|"+str(dtype)+"\n"

a=StringIO(output)
b=pd.read_csv(a, sep="|")
writer = pd.ExcelWriter("Schemachanges.xlsx", engine = 'xlsxwriter')
b.to_excel(writer, sheet_name=new+"removedfields", index=False)

Table|Column|DType
GCCT_EXECUTIVE|GCCT_COUNTRYMARKETIDNAME|NVARCHAR(155)
GCCT_QAMONITORING|OWNINGUSER|UNIQUEIDENTIFIER
GCCT_QAMONITORING|GCCT_QAAGENTIDNAME|NVARCHAR(200)
GCCT_QAMONITORING|CREATEDONBEHALFBYYOMINAME|NVARCHAR(100)
GCCT_FULFILLMENTVENDOR|GCCT_FULFILLMENTVENDOR_ROWVERSION|TIMESTAMP
GCCT_CASEDISPOSITIONTYPE|GCCT_CASEDISPOSITIONTYPE_IDENTITY|INT
GCCT_EXECUTIVE|GCCT_EXECUTIVEADMIN|NVARCHAR(100)
GCCT_GENERICINFORMATIONTOPIC|GCCT_GENERICINFORMATIONTOPIC_ROWVERSION|TIMESTAMP
CXLVHLP_CHATACTIVITY|CXLVHLP_CHATACTIVITY_IDENTITY|INT
GCCT_EXECUTIVE|GCCT_REGIONIDNAME|NVARCHAR(155)
GCCT_COACHBACK|GCCT_COACHBACK_ROWVERSION|TIMESTAMP
GCCT_FULFILLMENTVENDOR|GCCT_FULFILLMENTVENDOR_IDENTITY|INT
GCCT_EXECUTIVE|CREATEDBYNAME|NVARCHAR(100)
GCCT_PRIORDEALERDECISION|GCCT_PRIORDEALERDECISION_IDENTITY|INT
GCCT_EXECUTIVE|STATUSCODE|INT
GCCT_EXECUTIVE|MODIFIEDONBEHALFBYNAME|NVARCHAR(100)
GCCT_QAMONITORING|STATECODE|INT
GCCT_EXECUTIVE|OWNERIDTYPE|NVARCHAR(4000)
GCCT_ACCOUNTRESPONSIBLEAGENT|GCCT_ACCOUN

In [25]:
old_ddl=DESDSC_Jan25
new_ddl=prod
output=''
output=output+"Table|Column|DType\n"
#Dtype changes
for i in set(old_ddl[0]).intersection(set(new_ddl[0])):
    #print i
    old=old_ddl[old_ddl[0] == i].values[0][1]
    new=new_ddl[new_ddl[0] == i].values[0][1]
    if old!=new:
        print(str(prod[prod[0] == i].values[0][0]).replace("-","|").strip().replace(" ","|")+"|"+old+"|"+new+"\n")


a=StringIO(output)
b=pd.read_csv(a, sep="|")
writer = pd.ExcelWriter("Schemachanges.xlsx", engine = 'xlsxwriter')
b.to_excel(writer, sheet_name=new+"data_type_changes", index=False)

Table|Column|old|New
GCCT_BUYBACKEVALUATIONMILESTONES|GCCT_BUYBACKEVALUATIONMILESTONES_IDENTITY|INT|BIGINT
GCCT_LOYALTYASSISTANCE|GCCT_LOYALTYASSISTANCE_IDENTITY|INT|BIGINT
EMAIL|EMAIL_IDENTITY|INT|BIGINT
GCCT_ENGINE|GCCT_ENGINE_IDENTITY|INT|BIGINT
ACCOUNT|ACCOUNT_IDENTITY|INT|BIGINT
MSDYN_SURVEYINVITE|MSDYN_ORIGINATINGEMAILNAME|NVARCHAR(200)|NVARCHAR(400)
GCCT_ARBITRATIONCLAIMPROCESSING|GCCT_ARBITRATIONCLAIMPROCESSING_IDENTITY|INT|BIGINT
GCCT_GOODFAITHREVIEW|GCCT_GOODFAITHREVIEW_IDENTITY|INT|BIGINT
GCCT_CASEASSIGNMENT|GCCT_CASEASSIGNMENT_IDENTITY|INT|BIGINT
BUSINESSUNIT|BUSINESSUNIT_IDENTITY|INT|BIGINT
GCCT_DOAPROGRAMCODE|GCCT_DOAPROGRAMCODE_IDENTITY|INT|BIGINT
GCCT_PARTSORDERSTATUS|GCCT_PARTSORDERSTATUS_IDENTITY|INT|BIGINT
ACTIVITYPARTY|ACTIVITYPARTY_IDENTITY|INT|BIGINT
GCCT_ADDPORTALMESSAGE|GCCT_ADDPORTALMESSAGE_IDENTITY|INT|BIGINT
GCCT_FIELDINVOLVEMENTASSISTANCEREQUEST|GCCT_FIELDINVOLVEMENTASSISTANCEREQUEST_IDENTITY|INT|BIGINT
GCCT_MATERIALREQUESTDETAIL|GCCT_MATERIALREQUESTDETAIL_I